Run this notebook locally, with a .env containing your google API key GOOGLE_API_KEY = your key

Uncomment and run the cell below to install the dependencies

In [4]:
# pip install -r requirements.txt

In [5]:
# Imports
import os
from dotenv import load_dotenv
import psutil
import platform
from datetime import datetime
import logging, time, asyncio, json
import subprocess
from typing import Dict, Any
import shutil
import re

from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.adk.sessions import InMemorySessionService
from google.genai import types

In [6]:
# Load variables from .env file
load_dotenv()

# Access the key
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
print("✅ Gemini API key loaded:", bool(GOOGLE_API_KEY))

# Setup logging
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger("OptimAIzer")

✅ Gemini API key loaded: True


In [ ]:
# Configure Retry Options
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

## System Monitoring Tools (Custom Tools)

In [8]:
def get_cpu_usage() -> dict:
    """Get CPU usage per core and overall. Efficiently computes overall as average."""
    percents = psutil.cpu_percent(interval=1, percpu=True)
    data = {
        "overall_usage_percent": round(sum(percents) / len(percents), 1),
        "per_core_usage": percents,
        "timestamp": datetime.now().isoformat()
    }
    return data


def get_memory_usage() -> dict:
    """Get memory usage stats."""
    mem = psutil.virtual_memory()
    return {
        "total_gb": round(mem.total / (1024**3), 2),
        "used_gb": round(mem.used / (1024**3), 2),
        "usage_percent": mem.percent,
        "timestamp": datetime.now().isoformat()
    }


def get_disk_usage() -> dict:
    """Get disk usage per partition."""
    partitions = []
    for partition in psutil.disk_partitions():
        try:
            usage = psutil.disk_usage(partition.mountpoint)
            partitions.append({
                "device": partition.device,
                "total_gb": round(usage.total / (1024**3), 2),
                "free_gb": round(usage.free / (1024**3), 2),
                "usage_percent": usage.percent
            })
        except Exception:
            continue
    return {"partitions": partitions, "timestamp": datetime.now().isoformat()}


def get_top_processes() -> dict:
    """Get top 5 processes by CPU and Memory usage."""
    processes = []
    for proc in psutil.process_iter(['pid','name','cpu_percent','memory_percent']):
        try:
            if proc.info['name'].lower() == "system idle process":
                continue  # skip idle process
            processes.append(proc.info)
        except Exception:
            continue
    
    top_cpu = sorted(processes, key=lambda p: p['cpu_percent'] or 0, reverse=True)[:5]
    top_mem = sorted(processes, key=lambda p: p['memory_percent'] or 0, reverse=True)[:5]
    return {"top_cpu": top_cpu, "top_memory": top_mem, "timestamp": datetime.now().isoformat()}


def get_gpu_usage() -> dict:
    """Query GPU via nvidia-smi if available. Fallback gracefully if not."""
    try:
        out = subprocess.check_output(
            ["nvidia-smi", "--query-gpu=utilization.gpu,memory.used,memory.total",
             "--format=csv,noheader,nounits"],
            stderr=subprocess.DEVNULL
        ).decode().strip()
        gpus = []
        if out:
            for line in out.splitlines():
                util, mem_used, mem_total = [x.strip() for x in line.split(",")]
                gpus.append({
                    "utilization_percent": float(util),
                    "memory_used_mb": float(mem_used),
                    "memory_total_mb": float(mem_total)
                })
        return {"gpus": gpus, "timestamp": datetime.now().isoformat()}
    except Exception:
        return {"gpus": [], "note": "nvidia-smi not available", "timestamp": datetime.now().isoformat()}


def get_network_usage() -> dict:
    """Return network I/O counters."""
    try:
        net = psutil.net_io_counters()
        return {
            "bytes_sent": net.bytes_sent,
            "bytes_recv": net.bytes_recv,
            "packets_sent": net.packets_sent,
            "packets_recv": net.packets_recv,
            "timestamp": datetime.now().isoformat()
        }
    except Exception as e:
        return {"error": str(e), "timestamp": datetime.now().isoformat()}


def get_battery() -> dict:
    """Return battery status if available."""
    try:
        batt = psutil.sensors_battery()
        if batt is None:
            return {"available": False, "timestamp": datetime.now().isoformat()}
        return {
            "available": True,
            "percent": batt.percent,
            "plugged_in": bool(batt.power_plugged),
            "timestamp": datetime.now().isoformat()
        }
    except Exception:
        return {"available": False, "timestamp": datetime.now().isoformat()}


def get_temperatures() -> dict:
    """Return temperature sensors as floats."""
    try:
        temps = psutil.sensors_temperatures()
        temps_clean = {}
        for k, v in temps.items():
            temps_clean[k] = [round(float(t.current), 1) for t in v if hasattr(t, "current")]
        return {"temperatures": temps_clean, "timestamp": datetime.now().isoformat()}
    except Exception:
        return {"temperatures": {}, "timestamp": datetime.now().isoformat()}


def get_system_info() -> str:
    """Combine all monitoring tools into one JSON string for agents or reporting."""
    try:
        info = {
            "platform": platform.system(),
            "processor": platform.processor(),
            "ram_gb": round(psutil.virtual_memory().total / (1024.0 **3), 2),
            "cpu_cores": psutil.cpu_count(logical=False),
            "cpu": get_cpu_usage(),
            "memory": get_memory_usage(),
            "disk": get_disk_usage(),
            "top_processes": get_top_processes(),
            "gpu": get_gpu_usage(),
            "network": get_network_usage(),
            "battery": get_battery(),
            "temperatures": get_temperatures(),
            "timestamp": datetime.now().isoformat()
        }
        return json.dumps(info, indent=2)
    except Exception as e:
        return json.dumps({"error": str(e), "timestamp": datetime.now().isoformat()}, indent=2)


print("✅ Monitoring tools ready")

✅ Monitoring tools ready


## 2. Auto-Fix Tools (Actions)

In [ ]:
# Global OS type
OS_TYPE = platform.system()

# Cross-platform auto-fix tools

def restart_search_service() -> dict:
    """Restart system search/indexing service cross-platform."""
    try:
        if OS_TYPE == "Windows":
            subprocess.run(['net', 'stop', 'WSearch'], capture_output=True, shell=True)
            result = subprocess.run(['net', 'start', 'WSearch'], capture_output=True, shell=True)
            return {
                "status": "SUCCESS" if result.returncode == 0 else "FAILED",
                "message": "Windows Search restarted" if result.returncode == 0 else "Requires admin rights",
                "timestamp": datetime.now().isoformat()
            }

        elif OS_TYPE == "Darwin": # macOS
            subprocess.run(['launchctl', 'kickstart', '-k', 'system/com.apple.metadata.mds'], capture_output=True)
            return {"status": "SUCCESS", "message": "macOS Spotlight indexing restarted", "timestamp": datetime.now().isoformat()}

        elif OS_TYPE == "Linux":
            # tracker3 is common in GNOME; other systems may differ
            result = subprocess.run(['systemctl', 'restart', 'tracker-miner-fs-3'], capture_output=True)
            if result.returncode == 0:
                return {"status": "SUCCESS", "message": "Linux tracker indexer restarted", "timestamp": datetime.now().isoformat()}
            else:
                return {"status": "UNAVAILABLE", "message": "Linux indexer not available on this system", "timestamp": datetime.now().isoformat()}

        else:
            return {"status": "UNSUPPORTED", "message": f"Unsupported OS: {OS_TYPE}", "timestamp": datetime.now().isoformat()}

    except Exception as e:
        return {"status": "ERROR", "message": str(e), "timestamp": datetime.now().isoformat()}


def clear_temp_files() -> dict:
    """Clear temporary files in a cross-platform way."""
    try:
        temp_paths = []
        if OS_TYPE == "Windows":
            temp_paths = [os.environ.get("TEMP"), os.environ.get("TMP")]
        else:
            temp_paths = ["/tmp", "/var/tmp"]

        total_freed = 0
        for path in temp_paths:
            if path and os.path.exists(path):
                for filename in os.listdir(path):
                    try:
                        fpath = os.path.join(path, filename)
                        if os.path.isfile(fpath):
                            total_freed += os.path.getsize(fpath)
                            os.unlink(fpath)
                    except Exception:
                        continue  # skip files we cannot delete

        return {"status": "SUCCESS", "freed_mb": round(total_freed / (1024*1024), 2), "timestamp": datetime.now().isoformat()}

    except Exception as e:
        return {"status": "ERROR", "message": str(e), "timestamp": datetime.now().isoformat()}


def optimize_memory() -> dict:
    """Cross-platform memory optimization (safe)."""
    try:
        if OS_TYPE == "Windows":
            subprocess.run(['rundll32.exe', 'advapi32.dll,ProcessIdleTasks'], shell=True, capture_output=True)
            return {"status": "SUCCESS", "message": "Windows memory optimization triggered", "timestamp": datetime.now().isoformat()}

        elif OS_TYPE == "Darwin":
            result = subprocess.run(['sudo', 'purge'], capture_output=True)
            return {
                "status": "SUCCESS" if result.returncode == 0 else "FAILED",
                "message": "macOS purge executed" if result.returncode == 0 else "macOS purge requires admin rights",
                "timestamp": datetime.now().isoformat()
            }

        elif OS_TYPE == "Linux":
            # Safe suggestion instead of auto-executing
            return {
                "status": "TIP",
                "message": "Linux memory cleanup tip: sudo sh -c 'sync; echo 3 > /proc/sys/vm/drop_caches'",
                "note": "Not executed automatically for safety",
                "timestamp": datetime.now().isoformat()
            }

        else:
            return {"status": "UNSUPPORTED", "message": f"Unsupported OS: {OS_TYPE}", "timestamp": datetime.now().isoformat()}

    except Exception as e:
        return {"status": "ERROR", "message": str(e), "timestamp": datetime.now().isoformat()}


def analyze_process(process_name: str) -> dict:
    """Return detailed info for all processes matching name."""
    try:
        matches = []
        for proc in psutil.process_iter(['pid', 'name', 'cpu_percent', 'memory_percent']):
            if proc.info['name'] and proc.info['name'].lower() == process_name.lower():
                matches.append(proc.info)

        if matches:
            return {"status": "FOUND", "processes": matches, "timestamp": datetime.now().isoformat()}
        else:
            return {"status": "NOT_FOUND", "message": f"Process '{process_name}' not found", "timestamp": datetime.now().isoformat()}

    except Exception as e:
        return {"status": "ERROR", "message": str(e), "timestamp": datetime.now().isoformat()}


print("✅ Cross-platform auto-fix tools loaded successfully!")

✅ Cross-platform auto-fix tools loaded successfully!


## 3. Agent Creation (with the custom tools)

In [10]:
# Shared model
gemini_model = Gemini(model="gemini-2.5-flash-lite", api_key=GOOGLE_API_KEY, retry_options=retry_config)

# Monitor Agent
monitor_agent = Agent(
    model=gemini_model,
    name="monitor",
    instruction="""You are a system monitor. Call get_system_info() which returns everything:
        - CPU usage
        - Memory usage  
        - Disk usage
        - Top processes
        - GPU
        - Network stats
        - Battery
        - Temperatures
        Summarize the results in plain English. Flag any issues:
        - CPU > 80%
        - Memory > 85%
        - Disk > 90%
        - High resource processes""",
    tools=[get_system_info],
)

# Analyzer Agent
analyzer_agent = Agent(
    model=gemini_model,
    name="analyzer",
    instruction="""You are a system performance analyzer:
            Analyze system data. Identify root causes, assess severity (Critical/High/Medium/Low), and explain impact (You don't ask questions or wait for responses). 
            Focus on CPU, memory, disk, and process issues.""",
    tools=[analyze_process],
)

# Dynamic Auto-Fix Agent
autofix_agent = Agent(
    model=gemini_model,
    name="autofix",
    instruction="""You are a dynamic auto-fix agent. For ANY issue on this system:
        1. Use google_search to research unfamiliar problems
        2. Apply automated fixes using available tools
        3. Provide manual steps when automation isn't possible
        4. Be adaptive and research what you don't know
    Available fixes: restart_search_service, clear_temp_files, optimize_memory
    Safety: Never execute destructive commands. Always explain what each fix does.""",
    tools=[restart_search_service, clear_temp_files, optimize_memory, analyze_process, google_search],
)

print("✅ All agents created")

✅ All agents created


In [11]:
# Memory System
class SystemMemory:
    """Persistent memory - saves to disk"""
    
    def __init__(self, storage_file="optimaizer_history.json"):
        self.storage_file = storage_file
        self.history = []
        self._load_history()
    
    def _load_history(self):
        """Load history from disk"""
        if os.path.exists(self.storage_file):
            try:
                with open(self.storage_file, 'r') as f:
                    self.history = json.load(f)
                logger.info(f"Loaded {len(self.history)} past scans from disk")
            except:
                logger.warning("Could not load history file")
                self.history = []
        else:
            logger.info("No history file found - starting fresh")
    
    def record_scan(self, scan_data):
        """Save scan and persist to disk"""
        
        # Create a serializable version
        record = {
            "timestamp": datetime.now().isoformat(),
            "memory_pct": self._extract_memory(scan_data.get('monitor', '')),
            "cpu_pct": self._extract_cpu(scan_data.get('monitor', '')),
            "top_processes": self._extract_processes(scan_data.get('monitor', '')),
            "issues_found": 'critical' in scan_data.get('analysis', '').lower() or 'high' in scan_data.get('analysis', '').lower()
        }
        
        self.history.append(record)
        
        # Keep only last 30 scans to avoid huge files
        if len(self.history) > 30:
            self.history = self.history[-30:]
        
        # Save to disk
        self._save_history()
        logger.info(f"Scan recorded. Total history: {len(self.history)} scans")
    
    def _save_history(self):
        """Persist history to disk"""
        try:
            with open(self.storage_file, 'w') as f:
                json.dump(self.history, f, indent=2)
        except Exception as e:
            logger.error(f"Failed to save history: {e}")
    
    def _extract_memory(self, monitor_text):
        import re
        match = re.search(r'Memory.*?(\d+\.?\d*)%', monitor_text, re.IGNORECASE)
        return float(match.group(1)) if match else 0.0
    
    def _extract_cpu(self, monitor_text):
        import re
        match = re.search(r'CPU.*?(\d+\.?\d*)%', monitor_text, re.IGNORECASE)
        return float(match.group(1)) if match else 0.0
    
    def _extract_processes(self, monitor_text):
        import re
        # Extract process names
        processes = re.findall(r'([A-Za-z]+)\.exe', monitor_text, re.IGNORECASE)
        return list(set([p.lower() for p in processes]))[:5]
    
    def get_recent_scans(self, limit=5):
        """Get recent scan history"""
        return self.history[-limit:]
    
    def get_stats(self):
        """Get statistics across all scans"""
        if not self.history:
            return None
        
        mem_values = [s.get('memory_pct', 0) for s in self.history]
        cpu_values = [s.get('cpu_pct', 0) for s in self.history]
        
        return {
            "total_scans": len(self.history),
            "avg_memory": sum(mem_values) / len(mem_values),
            "max_memory": max(mem_values),
            "avg_cpu": sum(cpu_values) / len(cpu_values),
            "issues_count": sum(1 for s in self.history if s.get('issues_found', False))
        }

# Recreate memory with persistence
memory = SystemMemory()
print(f"✅ Persistent memory loaded ({len(memory.history)} past scans)")

2025-11-30 21:42:29,697 - INFO - Loaded 6 past scans from disk


✅ Persistent memory loaded (6 past scans)


## Orchestrator

In [12]:
# Main Orchestrator with Smart Summarization
class OptimAIzer:
    """Main orchestrator - coordinates all agents"""
    
    def __init__(self, monitor, analyzer, autofix, memory):
        self.monitor = monitor
        self.analyzer = analyzer
        self.autofix = autofix
        self.memory = memory
    
    def _extract_text(self, responses):
        """Extract text from agent responses"""
        text = ""
        for event in responses:
            if hasattr(event, 'content') and event.content:
                if hasattr(event.content, 'parts') and event.content.parts:
                    for part in event.content.parts:
                        if hasattr(part, 'text') and part.text:
                            text = part.text
        return text
    
    def _summarize_monitor_data(self, monitor_text):
        """Parse monitor JSON, fallback to text if necessary"""
        summary = {"raw_data": monitor_text}
        try:
            parsed = json.loads(monitor_text)
            summary.update({
                "cpu_usage": parsed.get("cpu", {}).get("overall_usage_percent"),
                "memory_usage": parsed.get("memory", {}).get("usage_percent"),
                "disk": parsed.get("disk"),
                "gpu": parsed.get("gpu", {}).get("utilization_percent") if parsed.get("gpu") else None,
                "network": parsed.get("network"),
                "battery": parsed.get("battery"),
                "temperatures": parsed.get("temperatures"),
            })
        except:
            pass
        return summary
    
    async def run(self, apply_fixes=False):
        """Run complete scan with optional auto-fix"""
        logger.info("Starting scan...")
        
        # Step 1: Monitor
        logger.info("Step 1: Monitoring system...")
        monitor_runner = InMemoryRunner(agent=self.monitor)
        monitor_resp = await monitor_runner.run_debug("Get complete system status with top 5 processes by CPU and memory")
        monitor_text = self._extract_text(monitor_resp)
        
        # Step 2: Analyze (pass full data to analyzer)
        logger.info("Step 2: Analyzing issues...")
        analyzer_runner = InMemoryRunner(agent=self.analyzer)
        
        # Create a structured prompt instead of dumping raw text
        analysis_prompt = f"""Analyze this system:

{monitor_text}

Identify:
1. Critical issues (anything above threshold)
2. Root causes
3. Severity levels
4. Which processes are responsible"""
        
        analyzer_resp = await analyzer_runner.run_debug(analysis_prompt)
        analysis_text = self._extract_text(analyzer_resp)
        
        # Step 3: Auto-Fix (if enabled)
        fix_text = "Auto-fix disabled (run with apply_fixes=True to enable)"
        if apply_fixes:
            logger.info("Step 3: Applying fixes...")
            autofix_runner = InMemoryRunner(agent=self.autofix)
            
            fix_prompt = f"""Based on this analysis, apply fixes:

{analysis_text}

Use your tools to:
1. Research any unfamiliar issues
2. Apply automated fixes where safe
3. Provide manual instructions for complex issues"""
            
            fix_resp = await autofix_runner.run_debug(fix_prompt)
            fix_text = self._extract_text(fix_resp)
        
        result = {
            "timestamp": datetime.now().isoformat(),
            "monitor": monitor_text,
            "analysis": analysis_text,
            "fixes": fix_text,
            "fixes_applied": apply_fixes
        }
        
        self.memory.record_scan(result)
        logger.info("Scan complete!")
        return result

# Recreate OptimAIzer
guard = OptimAIzer(monitor_agent, analyzer_agent, autofix_agent, memory)
print("✅ OptimAIzer ready!")

✅ OptimAIzer ready!


In [13]:
def generate_user_report(result: Dict[str, Any]) -> str:
    """System health report generator"""

    monitor = result.get("monitor", "") or ""

    report = []
    report.append("╔" + "=" * 78 + "╗")
    report.append("║" + " " * 20 + "OPTIMAIZER HEALTH REPORT" + " " * 33 + "║")
    report.append("╚" + "=" * 78 + "╝\n")

    # ------------------------------------------------
    # Flexible metric extraction (multi-pattern)
    # ------------------------------------------------

    def find_first(patterns):
        for pat in patterns:
            m = re.search(pat, monitor, re.IGNORECASE)
            if m:
                return m.group(1)
        return None

    mem_raw = find_first([
        r"(?:memory|ram)\s*usage[^%]*?(\d+\.?\d*)%",
        r"total\s*memory[^%]*?(\d+\.?\d*)%",
    ])

    cpu_raw = find_first([
        r"cpu\s*usage[^%]*?(\d+\.?\d*)%",
        r"total\s*cpu[^%]*?(\d+\.?\d*)%",
    ])

    gpu_raw = find_first([
        r"gpu\s*(?:load|utilization|usage)[^%]*?(\d+\.?\d*)%",
        r"video\s*engine[^%]*?(\d+\.?\d*)%",
        r"graphics[^%]*?(\d+\.?\d*)%",
    ])

    batt_raw = find_first([
        r"battery\s*(?:level|charge|capacity)?[^%]*?(\d+)\s*%",
        r"battery[^0-9%]*(\d+)\s*percent",
    ])

    mem_pct = float(mem_raw) if mem_raw else None
    cpu_pct = float(cpu_raw) if cpu_raw else None
    gpu_pct = float(gpu_raw) if gpu_raw else None
    batt_pct = int(batt_raw) if batt_raw else None

    # -----------------
    # Health Assessment
    # -----------------

    report.append("🏥 OVERALL HEALTH STATUS")
    report.append("-" * 80)

    if mem_pct is None or cpu_pct is None:
        report.append("⚠️ PARTIAL DATA - Metrics unavailable")
    elif mem_pct < 80 and cpu_pct < 80:
        report.append("✅ HEALTHY - Your system is running well!")
    elif mem_pct >= 85 or cpu_pct >= 90:
        report.append("🔴 CRITICAL - Immediate attention needed")
    else:
        report.append("⚠️ NEEDS ATTENTION - Monitor system performance")

    report.append("")

    # ----------- QUICK STATS -----------

    report.append("📊 QUICK STATS")
    report.append("-" * 80)

    report.append(f"💾 Memory: {mem_pct:.1f}%" if mem_pct is not None else "💾 Memory: Unavailable")
    report.append(f"⚡ CPU: {cpu_pct:.1f}%" if cpu_pct is not None else "⚡ CPU: Unavailable")
    report.append(f"🎮 GPU: {gpu_pct:.1f}%" if gpu_pct is not None else "🎮 GPU: Unavailable")

    if batt_pct is not None:
        report.append(f"🔋 Battery: {batt_pct}%")
    else:
        report.append("🔋 Battery: Unavailable")

    report.append("")

    # ----------------------------
    # PROCESS PARSING
    # ----------------------------

    proc_patterns = [
        r'([A-Za-z0-9_\-.\/]+)\s*[-:]\s*(\d+\.?\d*)%\s*(CPU)',
        r'([A-Za-z0-9_\-.\/]+)\s*[-:]\s*(\d+\.?\d*)%\s*(?:RAM|memory)',
    ]

    mem_dict = {}
    cpu_dict = {}

    for pat in proc_patterns:
        matches = re.findall(pat, monitor, re.IGNORECASE)

        for row in matches:
            name = row[0]
            pct = row[1]
            label = row[2].lower() if len(row) > 2 else "memory"

            key = name.lower().replace(".exe", "")
            val = float(pct)

            if "cpu" in label:
                cpu_dict[key] = max(cpu_dict.get(key, 0), val)
            else:
                mem_dict[key] = max(mem_dict.get(key, 0), val)

    top_mem = sorted(mem_dict.items(), key=lambda x: x[1], reverse=True)[:5]
    top_cpu = sorted(cpu_dict.items(), key=lambda x: x[1], reverse=True)[:5]

    # -----------------
    # Process Reporting
    # -----------------

    report.append("🔥 TOP PROCESSES")
    report.append("-" * 80)

    if not top_mem and not top_cpu:
        report.append("• No individual processes exceeded reporting thresholds.")
    else:
        for n, p in top_mem:
            report.append(f"💾 {n}: {p:.1f}% RAM")

        for n, p in top_cpu:
            report.append(f"⚡ {n}: {p:.1f}% CPU")

    report.append("")

    # -----------------
    # Recommendations
    # -----------------

    report.append("💡 RECOMMENDATIONS")
    report.append("-" * 80)

    suggested = False

    if mem_pct is not None and mem_pct >= 80 and top_mem:
        names = ", ".join(n for n, _ in top_mem[:3])
        report.append(f"• Close or restart memory-heavy apps: {names}")
        suggested = True

    if cpu_pct is not None and cpu_pct >= 80 and top_cpu:
        names = ", ".join(n for n, _ in top_cpu[:3])
        report.append(f"• Investigate high CPU usage from: {names}")
        suggested = True

    if not suggested:
        report.append("• No immediate action required.")

    report.append("")
    report.append("╚" + "=" * 78 + "╝")

    return "\n".join(report)


## Display report

In [14]:
# Run and Display User-Friendly Report
print("Running system check...\n")

result = await guard.run(apply_fixes=False)

# Generate and display clean report
user_report = generate_user_report(result)
print(user_report)

# Show technical details
print("\n\n" + "="*80)
print("📋 TECHNICAL DETAILS")
print("="*80)
print("\n🔍 RAW ANALYSIS:")
print(result['analysis'])

2025-11-30 21:42:29,736 - INFO - Starting scan...
2025-11-30 21:42:29,736 - INFO - Step 1: Monitoring system...


Running system check...


 ### Created new session: debug_session_id

User > Get complete system status with top 5 processes by CPU and memory


2025-11-30 21:42:30,411 - INFO - Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-11-30 21:42:31,474 - INFO - Response received from the model.
2025-11-30 21:42:31,476 - WARNING - Warning: there are non-text parts in the response: ['function_call'], returning concatenated text result from text parts. Check the full candidates.content.parts accessor to get the full model response.
2025-11-30 21:42:34,305 - INFO - Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-11-30 21:42:37,914 - INFO - Response received from the model.
2025-11-30 21:42:37,917 - INFO - Step 2: Analyzing issues...
2025-11-30 21:42:37,919 - INFO - Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False


monitor > The system CPU usage is at 8.4%, which is normal. However, the memory usage is at 90.4%, exceeding the 85% threshold. The disk usage is at 58.7%, which is normal.

The top 5 processes by CPU usage are:
- System (PID 4): 0.0% CPU, 0.000801290028382664% Memory
- (PID 284): 0.0% CPU, 0.4002322284191349% Memory
- Registry (PID 328): 0.0% CPU, 0.33399225273950134% Memory
- Notepad.exe (PID 556): 0.0% CPU, 0.247792870898336% Memory
- smss.exe (PID 904): 0.0% CPU, 0.0014083279286725612% Memory

The top 5 processes by memory usage are:
- chrome.exe (PID 53188): 0.0% CPU, 6.287747134234777% Memory
- chrome.exe (PID 29456): 0.0% CPU, 5.485922912499857% Memory
- chrome.exe (PID 24396): 0.0% CPU, 4.079488942076201% Memory
- Code.exe (PID 57636): 0.0% CPU, 3.8805504813931955% Memory
- chrome.exe (PID 54168): 0.0% CPU, 3.4129856090739055% Memory

**Issue:** High memory usage (90.4%). Several chrome.exe processes are consuming a significant amount of memory.

 ### Created new session: debug

2025-11-30 21:42:39,466 - INFO - Response received from the model.
2025-11-30 21:42:39,474 - INFO - Scan recorded. Total history: 7 scans
2025-11-30 21:42:39,476 - INFO - Scan complete!


analyzer > **Analysis Results:**

**1. Critical Issues:**
*   **High Memory Usage:** Memory utilization is at 90.4%, exceeding the 85% threshold.

**2. Root Causes:**
*   The primary driver of high memory usage appears to be multiple instances of `chrome.exe`, collectively consuming a substantial portion of the system's memory.

**3. Severity Levels:**
*   **Memory Usage:** Critical (above 85% threshold)

**4. Processes Responsible:**
*   **`chrome.exe` (PIDs: 53188, 29456, 24396, 54168):** These processes are the main contributors to the high memory consumption.
*   **`Code.exe` (PID: 57636):** Also a significant contributor to memory usage.
╔==============================================================================╗
║                    OPTIMAIZER HEALTH REPORT                                 ║
╚==============================================================================╝

🏥 OVERALL HEALTH STATUS
--------------------------------------------------------------------------------
🔴

## Apply fixes (OPTIONAL TO RUN - WILL MAKE CHANGES TO YOUR SYSTEM)

In [15]:
# Apply Fixes (OPTIONAL - Uncomment to use)
# ⚠️ WARNING: This will make real changes to your system
# Uncomment below to actually apply fixes:

# print("="*60)
# print("APPLYING AUTO-FIXES")
# print("="*60)
# 
# fix_result = await guard.run(apply_fixes=True)
# 
# print("\n✅ FIXES APPLIED:")
# print(fix_result['fixes'])

## Safe Testing Mode (Asks for confirmation before applying fixes)

In [16]:
# Safe Testing Mode
class SafeTestMode:
    """Test auto-fixes safely without making real changes"""
    
    def __init__(self, autofix_agent):
        self.autofix = autofix_agent
    
    async def test_fixes(self, issues_description: str):
        """Test what fixes would be applied without executing them"""
        
        print("="*80)
        print("🧪 SAFE TEST MODE - No Real Changes Will Be Made")
        print("="*80)
        
        test_prompt = f"""
You have these system issues:

{issues_description}

For EACH issue, explain:
1. What fix you would apply
2. Which tool you would use (restart_search_service, clear_temp_files, optimize_memory)
3. What the expected outcome is
4. Any risks or side effects

DO NOT actually call the tools - just explain what you WOULD do.
Be specific about the commands or actions.
"""
        
        runner = InMemoryRunner(agent=self.autofix)
        responses = await runner.run_debug(test_prompt)
        
        # Extract text
        result_text = ""
        for event in responses:
            if hasattr(event, 'content') and event.content:
                if hasattr(event.content, 'parts') and event.content.parts:
                    for part in event.content.parts:
                        if hasattr(part, 'text') and part.text:
                            result_text = part.text
        
        return result_text

# Create safe tester
safe_tester = SafeTestMode(autofix_agent)
print("✅ Safe test mode ready")

✅ Safe test mode ready


In [17]:
# Interactive Temp File Functions
def preview_temp_files() -> str:
    """Preview what would be deleted WITHOUT deleting anything"""
    try:
        temp_dirs = [os.environ.get('TEMP'), os.environ.get('TMP')]
        
        preview = {
            "folders_to_clean": [],
            "total_size_mb": 0,
            "total_files": 0,
            "sample_files": []
        }
        
        for temp_dir in temp_dirs:
            if not temp_dir or not os.path.exists(temp_dir):
                continue
            
            folder_size = 0
            folder_files = 0
            
            for filename in os.listdir(temp_dir)[:100]:
                try:
                    file_path = os.path.join(temp_dir, filename)
                    if os.path.isfile(file_path):
                        size = os.path.getsize(file_path)
                        folder_size += size
                        folder_files += 1
                        
                        if len(preview["sample_files"]) < 10:
                            preview["sample_files"].append({
                                "name": filename[:60],
                                "size_kb": round(size / 1024, 2),
                                "age_days": round((datetime.now().timestamp() - os.path.getmtime(file_path)) / (24*60*60), 1)
                            })
                except:
                    continue
            
            if folder_files > 0:
                preview["folders_to_clean"].append({
                    "path": temp_dir,
                    "files": folder_files,
                    "size_mb": round(folder_size / (1024*1024), 2)
                })
                preview["total_size_mb"] += round(folder_size / (1024*1024), 2)
                preview["total_files"] += folder_files
        
        return json.dumps(preview, indent=2)
    except Exception as e:
        return json.dumps({"error": str(e)})

def clear_temp_files_interactive() -> str:
    """Clear temp files with user confirmation"""
    try:
        preview = json.loads(preview_temp_files())
        
        if preview.get("total_size_mb", 0) < 1:
            return "✅ No significant temp files to clean"
        
        print("\n" + "="*80)
        print("🗑️  TEMP FILE CLEANUP PREVIEW")
        print("="*80)
        print(f"\n📊 Total: {preview['total_files']} files, {preview['total_size_mb']} MB")
        
        print("\n📁 Locations:")
        for folder in preview['folders_to_clean']:
            print(f"  • {folder['path']}: {folder['files']} files, {folder['size_mb']} MB")
        
        print("\n📄 Sample files:")
        for i, file in enumerate(preview['sample_files'][:5], 1):
            print(f"  {i}. {file['name']} ({file['size_kb']} KB, {file['age_days']} days old)")
        
        print("\n" + "="*80)
        print("ℹ️  Your Documents, Downloads, Photos are NOT affected")
        print("="*80)
        
        response = input("\n🤔 Delete these temp files? (y/n): ").strip().lower()
        
        if response not in ['yes', 'y']:
            return "❌ Cleanup cancelled. No files deleted."
        
        print("\n🔄 Deleting...")
        
        temp_dirs = [os.environ.get('TEMP'), os.environ.get('TMP')]
        deleted = 0
        freed = 0
        
        for temp_dir in temp_dirs:
            if not temp_dir or not os.path.exists(temp_dir):
                continue
            
            for filename in os.listdir(temp_dir):
                try:
                    file_path = os.path.join(temp_dir, filename)
                    if os.path.isfile(file_path):
                        size = os.path.getsize(file_path)
                        os.unlink(file_path)
                        deleted += 1
                        freed += size
                except:
                    continue
        
        return f"✅ Deleted {deleted} files, freed {freed/(1024*1024):.2f} MB"
        
    except Exception as e:
        return f"ERROR: {str(e)}"

print("✅ Interactive temp cleaner ready")

✅ Interactive temp cleaner ready


In [18]:
# Interactive OptimAIzer
class InteractiveOptimAIzer:
    """OptimAIzer with user confirmations"""
    
    def __init__(self, monitor, analyzer, memory):
        self.monitor = monitor
        self.analyzer = analyzer
        self.memory = memory
    
    def _extract_text(self, responses):
        text = ""
        for event in responses:
            if hasattr(event, 'content') and event.content:
                if hasattr(event.content, 'parts') and event.content.parts:
                    for part in event.content.parts:
                        if hasattr(part, 'text') and part.text:
                            text = part.text
        return text
    
    async def run_interactive(self):
        """Run with confirmations"""
        print("\n🔍 Scanning system...")
        monitor_runner = InMemoryRunner(agent=self.monitor)
        monitor_resp = await monitor_runner.run_debug("Get complete system status")
        monitor_text = self._extract_text(monitor_resp)
        
        print("🧠 Analyzing...")
        analyzer_runner = InMemoryRunner(agent=self.analyzer)
        analyzer_resp = await analyzer_runner.run_debug(f"Analyze: {monitor_text}")
        analysis_text = self._extract_text(analyzer_resp)
        
        # Check memory
        import re
        mem_match = re.search(r'Memory usage.*?(\d+\.?\d*)%', monitor_text, re.IGNORECASE)
        memory_pct = float(mem_match.group(1)) if mem_match else 0
        
        print("\n" + "="*80)
        print("💡 RECOMMENDED FIXES")
        print("="*80)
        
        fixes = []
        
        if memory_pct >= 84:
            print("\n1️⃣  OPTIMIZE MEMORY (Safe - no files deleted)")
            response = input("   Apply? (y/n): ").strip().lower()
            if response in ['yes', 'y']:
                fixes.append('optimize_memory')
        
        print("\n2️⃣  CLEAN TEMP FILES (Will show preview)")
        response = input("   Check temp files? (y/n): ").strip().lower()
        if response in ['yes', 'y']:
            fixes.append('clear_temp')
        
        if not fixes:
            print("\n✅ No fixes selected")
            return {"monitor": monitor_text, "analysis": analysis_text, "fixes_applied": False}
        
        print("\n" + "="*80)
        print("🔧 APPLYING FIXES")
        print("="*80)
        
        results = []
        
        for fix in fixes:
            if fix == 'optimize_memory':
                print("\n⚙️  Optimizing memory...")
                r = optimize_memory()
                print(f"   {r}")
                results.append(r)
            
            elif fix == 'clear_temp':
                r = clear_temp_files_interactive()
                results.append(r)
        
        print("\n✅ DONE")
        
        return {
            "monitor": monitor_text,
            "analysis": analysis_text,
            "fixes_applied": True,
            "fix_results": results
        }

interactive_guard = InteractiveOptimAIzer(monitor_agent, analyzer_agent, memory)
print("✅ Interactive OptimAIzer ready")

✅ Interactive OptimAIzer ready


In [19]:
result = await interactive_guard.run_interactive()

# Show report
print("\n")
print(generate_user_report(result))

2025-11-30 21:42:39,576 - INFO - Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False



🔍 Scanning system...

 ### Created new session: debug_session_id

User > Get complete system status


2025-11-30 21:42:40,072 - INFO - Response received from the model.
2025-11-30 21:42:41,992 - INFO - Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-11-30 21:42:44,623 - INFO - Response received from the model.
2025-11-30 21:42:44,627 - INFO - Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False


monitor > The system is currently experiencing high memory usage. The memory usage is at 89.2%, which is above the 85% threshold.

CPU usage is at 17.4%, which is normal.
Disk usage is at 58.7%, which is normal.

The top CPU-consuming processes are:
- svchost.exe (68.6% CPU)
- Code.exe (35.3% CPU)
- System (27.3% CPU)
- Code.exe (21.3% CPU)
- python.exe (14.2% CPU)

The top memory-consuming processes are:
- chrome.exe (5.88% memory)
- chrome.exe (5.42% memory)
- chrome.exe (4.07% memory)
- Code.exe (3.85% memory)
- chrome.exe (3.41% memory)

The GPU is being utilized at 4.0%, with 722.0 MB of memory used out of 8188.0 MB.
Network traffic shows 35168736 bytes sent and 401570715 bytes received.
The battery is plugged in and at 93% charge.
There are no temperature readings available.

**Issue:** High memory usage (89.2%).
🧠 Analyzing...

 ### Created new session: debug_session_id

User > Analyze: The system is currently experiencing high memory usage. The memory usage is at 89.2%, which i

2025-11-30 21:42:47,469 - INFO - Response received from the model.


analyzer > Analyzing the provided system data:

**High Memory Usage (Severity: High)**
*   **Root Cause:** While the specific process consuming the most memory isn't definitively pinpointed by the provided list (as the percentages are relative and cumulative), the high overall memory usage at 89.2% indicates a potential bottleneck. The top listed memory consumers are multiple instances of `chrome.exe` and `Code.exe`.
*   **Impact:** Sustained high memory usage can lead to system slowdowns, application unresponsiveness, and potential instability as the system struggles to allocate resources. If memory is exhausted, the system may resort to using the page file (virtual memory on disk), which is significantly slower and can further degrade performance.

**CPU Usage:**
*   **Status:** Normal (17.4%).
*   **Note:** Although overall CPU usage is normal, the listed top CPU-consuming processes (`svchost.exe`, `Code.exe`, `System`, `python.exe`) indicate specific processes are actively using CP

2025-11-30 21:42:49,151 - INFO - Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-11-30 21:42:53,133 - INFO - Response received from the model.


analyzer > Analyzing the processes:

**chrome.exe:**
*   Multiple instances of `chrome.exe` are running.
*   The highest memory consumer is `chrome.exe` with PID 29456, using 5.28% of memory.
*   Another instance, PID 54168, uses 3.41% of memory.
*   PID 24396 uses 4.05% of memory.
*   Several other instances use between 0.02% and 2.88% of memory.
*   CPU usage for `chrome.exe` instances is generally low, with a few exceptions (e.g., PID 28776 at 2.5%, PID 54168 at 2.9%, PID 29456 at 0%).

**Code.exe:**
*   Multiple instances of `Code.exe` are running.
*   The highest memory consumer is `Code.exe` with PID 57636, using 3.81% of memory.
*   PID 41636 uses 2.40% of memory and 13% CPU.
*   PID 10820 uses 0.64% memory and 9.3% CPU.
*   Other instances use between 0.06% and 0.85% of memory.

**Conclusion:**

The high memory usage appears to be primarily driven by numerous instances of `chrome.exe`, with several consuming significant amounts of memory individually. There are also a few insta